In [1]:
# import packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk  
import tensorflow as tf
!pip install transformers
import torch

# BERT and T5 are easily imported from Hugging Face's transformer library (https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel
from transformers import BertTokenizer, TFBertModel, T5Tokenizer,  MT5Tokenizer, TFT5ForConditionalGeneration
from torch.utils.data import TensorDataset, random_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

# Keras functional API
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import Input

!pip install pyspellchecker  
from spellchecker import SpellChecker
  
import more_itertools
from nltk import tokenize
nltk.download('punkt')

     |████████████████████████████████| 2.5MB 8.7MB/s 
     |████████████████████████████████| 901kB 33.1MB/s 
     |████████████████████████████████| 3.3MB 32.2MB/s 
     |████████████████████████████████| 2.7MB 9.2MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import transformers
#!pip install -q transformers

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [65]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_set.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')

In [74]:
def tokenized(df, examples):
    input_ids = []
    token_type_ids = [] 
    attention_masks = []
    window = 3
    for sentences in df['sent'][:examples]:
        if len(sentences) > window:
            i = list(more_itertools.windowed([k for k in range(len(sentences))],n=window, step=1))
        else:
            i = list(more_itertools.windowed([k for k in range(len(sentences))],n=len(sentences), step=1))
        w_sentences = []
        w = ''
        for hi in i:
            w = ''
            for j in hi:
                w = w + sentences[j] + ' '
            w_sentences.append(w)
        encoded_dict = tokenizer([x for x in w_sentences], 
              max_length=50,
              add_special_tokens = True,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
         # Add the encoded sentence to the list    
        input_ids.append(encoded_dict['input_ids'])
         # Add the encoded sentence to the list.    
        token_type_ids.append(encoded_dict['token_type_ids'])
         # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
    return input_ids, token_type_ids, attention_masks

def coherence(df, examples):
  df['sent'] = df['essay'].apply(lambda x: tokenize.sent_tokenize(x))
  z = tokenized(df, examples)
  score = []
  for i in range(examples):
    print(i)
    bert_inputs = {'input_ids': z[0][i],
                    'token_type_ids': z[1][i],
                    'attention_mask': z[2][i]}

    bert_out = bert_model(bert_inputs)
    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])
    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    classification = classification.numpy()
    cl_score = 0
    for cl in range(len(classification)):
      cl_score = cl_score + (-classification[cl]*np.log(classification[cl]) - (1-classification[cl])*np.log(1-classification[cl]))
    score.append(cl_score*(1/len(classification)))
  return score


In [82]:
coherence(train,100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


[array([0.68928677], dtype=float32),
 array([0.6836403], dtype=float32),
 array([0.5138546], dtype=float32),
 array([0.5252035], dtype=float32),
 array([0.5562195], dtype=float32),
 array([0.6773198], dtype=float32),
 array([0.6483237], dtype=float32),
 array([0.6803266], dtype=float32),
 array([0.6533462], dtype=float32),
 array([0.6185131], dtype=float32),
 array([0.3603744], dtype=float32),
 array([0.48444727], dtype=float32),
 array([0.6375341], dtype=float32),
 array([0.6855405], dtype=float32),
 array([0.6565869], dtype=float32),
 array([0.68985933], dtype=float32),
 array([0.60422033], dtype=float32),
 array([0.6721679], dtype=float32),
 array([0.5464715], dtype=float32),
 array([0.6875904], dtype=float32),
 array([0.683491], dtype=float32),
 array([0.6439939], dtype=float32),
 array([0.5352503], dtype=float32),
 array([0.65701693], dtype=float32),
 array([0.61611617], dtype=float32),
 array([0.22709155], dtype=float32),
 array([0.5425353], dtype=float32),
 array([0.68624145], d

In [28]:
max_length = 512
num_train_examples = 50

def training_pipeline(df):

    """
        Simplify dataframe and normalize scores based on essay set.
    """
    df = df[['essay_set','essay','rater1_domain1','rater2_domain1','domain1_score']]
    df = normalize(df)
    
   
    """
        Create features based on length, average/std of word length, unique words. 
    """
    
    df['length'] = df['essay'].str.split().str.len()
    df['avg_word_len'] = df['essay'].str.split().apply(lambda x: np.mean([len(y) for y in x]))
    df['std_word_len'] = df['essay'].str.split().apply(lambda x: np.std([len(y) for y in x]))
    df['unique_words'] = df['essay'].str.split().apply(lambda x: len(np.unique(x)))
    
    """
        Run first stage of network with BERT and predict output.
    """
    x_train = tokenize_function(df['essay'].tolist())
    y_train = df.normalized_score[:num_train_examples]
    fsm = first_stage_model()
    fsm.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask], y_train, epochs=5, batch_size=8)
    X_fsm = fsm.predict(x_train)
   
    """
        Combine output of first-stage model with other features for second stage model.
    """

    X_ext = np.array([X_fsm, df['length'], df['avg_word_len'], df['std_word_len'], df['unique_words']])


    """
        Run second stage model.
    """
    
    ssm = second_stage_model()
    ssm.fit(X_ext, y_train) 
    
    
    
def normalize(df):
    df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
    return df
    
def tokenize_function(example):
    return tokenizer([x for x in example[:num_train_examples]], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

def first_stage_model(hidden_size = 200, optimizer=tf.keras.optimizers.Adam()):
    
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

    
    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

    classification = tf.keras.layers.Dense(8, activation='softmax',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.MeanAbsoluteError(name='mean_absolute_error'),
                            metrics=['accuracy'])


    return classification_model
    
def second_stage_model():
    return DecisionTreeClassifier(random_state=0)

In [13]:
training_pipeline(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

KeyboardInterrupt: ignored

In [25]:
x_train = tokenize_function(train['essay'].tolist())


In [30]:
len(x_train.input_ids)

50

In [ ]:
train.head()

In [ ]:
x_train.input_ids[0]